# Document Classification (Spam filtering)

Reference: http://zacstewart.com/2015/04/28/document-classification-with-scikit-learn.html

Spam filtering is kind of like the "Hello world" of document classification. It's a binary classification problem: either spam, or not spam (a.k.a ham).

Download data from http://www2.aueb.gr/users/ion/data/enron-spam/

We'll work through to following tasks to get there:

- Loading raw email data into a workable format
- Extracting features from the raw data that an algorithm can learn from
- Training a classifier
- Evaluating accuracy by cross-validation
- Improving upon initial accuracy

# Loading Examples

According to protocol, email headers and bodies are separated by a blank line (NEWLINE), so we simply ignore all lines before that and then yield the rest of the email. You'll also notice the encoding="latin-1" bit. Some of the corpus is not in Unicode, so this makes a "best effort" attempt to decode the files correctly. 

(About function os.walk used in the following code:

os.walk(top, topdown=True, onerror=None, followlinks=False)
Generate the file names in a directory tree by walking the tree either top-down or bottom-up. For each directory in the tree rooted at directory top (including top itself), it yields a 3-tuple (dirpath, dirnames, filenames).

dirpath is a string, the path to the directory. dirnames is a list of the names of the subdirectories in dirpath (excluding '.' and '..'). filenames is a list of the names of the non-directory files in dirpath. Note that the names in the lists contain no path components. To get a full path (which begins with top) to a file or directory in dirpath, do os.path.join(dirpath, name). )

(About method str.join used in the following code:
It takes a list of strings and joins them, seprated by the first string.
For example ";".join(["A","B","C"]) is "A;B;C" .
)

In [1]:
import os

SKIP_FILES = {'cmds'}

def read_files(path):
    for root, dir_names, file_names in os.walk(path):
        for path in dir_names:
            read_files(os.path.join(root, path))
        for file_name in file_names:
            if file_name not in SKIP_FILES:
                file_path = os.path.join(root, file_name)
                past_header, lines = False, []
                f = open(file_path, encoding="latin-1")
                for line in f:
                    if past_header:
                        lines.append(line)
                    elif line == '\n':
                        past_header = True
                f.close()
                content = '\n'.join(lines)
                yield file_path, content

Now we need to build a dataset from all these email bodies. 

In [2]:
from pandas import DataFrame
import pandas as pd
import numpy as np

#### Build data frame of texts and labels:

In [3]:
rows = []
index = []
for file_path, text in read_files(".\\emails"):
    if file_path.find("emails\\spam") != -1:
        classification = 1
    else:
        classification = 0
    rows.append({'text': text, 'class': classification})
    index.append(file_path)

data_frame = DataFrame(rows, index=index)

In [4]:
data_frame.head()

,class,text
.\emails\ham\beck-s\2001_plan\1,0,"Guys, attached you will find a final cut on th..."
.\emails\ham\beck-s\2001_plan\2,0,I am still in need of the information regardin...
.\emails\ham\beck-s\2001_plan\3,0,Attached is a file containing all cost centers...
.\emails\ham\beck-s\2001_plan\4,0,"Sarah,\n\nBelow is our high level explanation ..."
.\emails\ham\beck-s\2001_plan\5,0,Sally - I will check into the meaning of this ...


In [5]:
data_frame = data_frame.reindex(np.random.permutation(data_frame.index))

In [6]:
data_frame.head()

,class,text
.\emails\spam\GP\part10\msg1271.eml,1,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 3.2//E..."
.\emails\spam\GP\part11\msg3650.eml,1,This is a multi-part message in MIME format.\n...
.\emails\spam\GP\part4\msg7242.eml,1,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 3.2//E..."
.\emails\spam\GP\part12\msg3805.eml,1,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 3.2//E..."
.\emails\spam\GP\part6\msg10215.eml,1,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.0 Tr..."


# Extracting Features

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer() #New object of class CountVectorizer
counts = count_vectorizer.fit_transform(data_frame['text'].values)

We'll instantiate a CountVectorizer and then call its instance method fit_transform, which does two things: it learns the vocabulary in all emails and extracts word count features. This method is an efficient way to do both steps, and for us it does the job. However, in some cases you may want to use a different vocabulary than the one inherent in the raw data. For this reason, CountVectorizer provides fit and transform methods to do them separately. Additionally, you can provide a vocabulary in the constructor.

#### Example:

In [8]:
dd = DataFrame([{"A": "hello world bye hello"}, {"A": "bye you" }])
print(dd)
result = CountVectorizer().fit_transform(dd['A'].values)
print(result)

                       A
0  hello world bye hello
1                bye you
  (0, 1)	2
  (0, 2)	1
  (0, 0)	1
  (1, 0)	1
  (1, 3)	1


# Classifying Emails (Naive Bayes)

In [9]:
from sklearn.naive_bayes import MultinomialNB

classifier = MultinomialNB()
targets = data_frame['class'].values
classifier.fit(counts, targets)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

And there we have it: a trained spam classifier. We can try it out by constructing some examples and predicting on them.

In [10]:
examples = ['Free Viagra call today!', "I'm going to attend the Linux users group tomorrow."]
example_counts = count_vectorizer.transform(examples)
print(example_counts)
predictions = classifier.predict(example_counts)
predictions # [1, 0]

  (0, 177631)	1
  (0, 266268)	1
  (0, 540915)	1
  (0, 570859)	1
  (1, 145540)	1
  (1, 282034)	1
  (1, 284997)	1
  (1, 372175)	1
  (1, 535695)	1
  (1, 540722)	1
  (1, 541321)	1
  (1, 560873)	1


array([1, 0], dtype=int64)

# Pipelining

Still, doing each one of those steps one-at-a-time was pretty tedious. We can package it all up using a construct provided by scikit-learn called a Pipeline.

A pipeline does exactly what it sounds like: connects a series of steps into one object which you train and then use to make predictions. In short, we can use a pipeline to merge the feature extraction and classification into one operation:

In [11]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('vectorizer',  CountVectorizer()),
    ('classifier',  MultinomialNB()) ])

pipeline.fit(data_frame['text'].values, data_frame['class'].values)
pipeline.predict(examples) # ['spam', 'ham']

array([1, 0], dtype=int64)

# Cross Validation

In [ ]:
from sklearn.cross_validation import KFold
from sklearn.metrics import confusion_matrix, f1_score

k_fold = KFold(n=len(data_frame), n_folds=6)
scores = []
confusion = np.array([[0, 0], [0, 0]])
for train_indices, test_indices in k_fold:
    train_text = data_frame.iloc[train_indices]['text'].values
    train_y = data_frame.iloc[train_indices]['class'].values

    test_text = data_frame.iloc[test_indices]['text'].values
    test_y = data_frame.iloc[test_indices]['class'].values

    pipeline.fit(train_text, train_y)
    predictions = pipeline.predict(test_text)

    confusion += confusion_matrix(test_y, predictions)
    score = f1_score(test_y, predictions, pos_label=1)
    scores.append(score)

print('Total emails classified:', len(data_frame))
print('Score:', sum(scores)/len(scores))
print('Confusion matrix:')
print(confusion)

# Total emails classified: 55326
# Score: 0.942661080942
# Confusion matrix:
# [[21660   178]
#  [ 3473 30015]]

scikit-learn provides various functions for evaluating the accuracy of a model. We're using the F1 score for each fold, which we then average together for a mean accuracy on the entire set. Using the model we just built and the example data sets mentioned in the beginning of this tutorial, we get about 0.94. A confusion matrix helps elucidate how the model did for individual classes. Out of 55,326 examples, we get about 178 false spams, and 3,473 false hams. I say "about" because by shuffling the data as we did, these numbers will vary each time we run the model.

That's really not bad for a first run. Obviously it's not production-ready even if we don't consider the scaling issues. Consumers demand accuracy, especially regarding false spams. Who doesn't hate to lose something important to the spam filter?

# Improving Results

In order to get better results, there's a few things we can change. We can try to extract more features from the emails, we can try different kinds of features, we can tune the parameters of the naïve Bayes classifier, or try another classifier all together.

One way to get more features is to use n-gram counts instead of just word counts. So far we've relied upon what's known as "bag of words" features. It's called that because we simply toss all the words of a document into a "bag" and count them, disregarding any meaning that could locked up in the ordering of words.

An n-gram can be thought of as a phrase that is n words long. For example, in the sentence "Don't tase me, bro" we have the 1-grams, "don't," "tase," "me," and "bro." The same sentence also has the 2-grams (or bigrams) "don't tase", "tase me", and "me bro." We can tell CountVectorizer to include any order of n-grams by giving it a range. For this data set, unigrams and bigrams seem to work well. 3-grams add a tiny bit more accuracy, but for the computation time they incur, it's probably not worth the marginal increase.

In [ ]:
pipeline = Pipeline([
    ('count_vectorizer', CountVectorizer(ngram_range=(1, 2))),
    ('classifier',       MultinomialNB())
])

scores = []
confusion = np.array([[0, 0], [0, 0]])
for train_indices, test_indices in k_fold:
    train_text = data_frame.iloc[train_indices]['text'].values
    train_y = data_frame.iloc[train_indices]['class'].values

    test_text = data_frame.iloc[test_indices]['text'].values
    test_y = data_frame.iloc[test_indices]['class'].values

    pipeline.fit(train_text, train_y)
    predictions = pipeline.predict(test_text)

    confusion += confusion_matrix(test_y, predictions)
    score = f1_score(test_y, predictions, pos_label=1)
    scores.append(score)

print('Total emails classified:', len(data_frame))
print('Score:', sum(scores)/len(scores))
print('Confusion matrix:')
print(confusion)

# Total emails classified: 55326
# Score: 0.978154601119
# Confusion matrix:
# [[21745    93]
#  [ 1343 32145]]

That boosts the model up to an F1 score of about 0.98. As before, it's a good idea to keep an eye on how it's doing for individual classes and not just the set as a whole. Luckily this increase represents an increase for both spam and ham classification accuracy.

Another way to improve accuracy is to use different kinds of features. N-gram counts have the disadvantage of unfairly weighting longer documents. A six-word spammy message and a five-page, heartfelt letter with six "spammy" words could potentially receive the same "spamminess" probability. To counter that, we can use frequencies rather than occurances. That is, focusing on how much of the document is made up of a particular word, instead of how many times the word appears in the document. This kind of feature set is known as term frequencies.

In addition to converting counts to frequencies, we can reduce noise in the features by reducing the weight for words that are common across the entire corpus. For example, words like "and," "the," and "but" probably don't contain a lot of information about the topic of the document, even though they will have high counts and frequencies across both ham and spam. To remedy that, we can use what's known as inverse document frequency or IDF.

--> See the reference for improvement.